In [3]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np

import optuna
from src import objective
# import sys
# sys.path.append('../')
# from src.objective import Objective

#### 再現のための乱数値指定

In [4]:
SEED = 334

#### 学習データの読み込み

In [5]:
boston = load_boston()
df_X = pd.DataFrame(boston.data, columns = boston.feature_names, dtype = float)
df_y = pd.Series(boston.target, name = 'PRICE', dtype = float)

#### データの分割

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state = SEED)

#### インスタンス化

In [8]:
objective = objective.Objective(RandomForestRegressor, X_train, y_train, random_state = SEED, cv = 5)

#### 最適化

In [9]:
study = optuna.create_study()
study.optimize(objective, n_trials=10)

[I 2020-07-24 21:17:26,938] Finished trial#0 with value: -0.8314347751255221 with parameters: {'min_samples_split': 15, 'max_depth': 97, 'n_estimators': 155}. Best is trial#0 with value: -0.8314347751255221.
[I 2020-07-24 21:17:30,793] Finished trial#1 with value: -0.8435614074574971 with parameters: {'min_samples_split': 5, 'max_depth': 117, 'n_estimators': 472}. Best is trial#1 with value: -0.8435614074574971.
[I 2020-07-24 21:17:33,316] Finished trial#2 with value: -0.8405350373971177 with parameters: {'min_samples_split': 10, 'max_depth': 391, 'n_estimators': 360}. Best is trial#1 with value: -0.8435614074574971.
[I 2020-07-24 21:17:33,789] Finished trial#3 with value: -0.8320037703866419 with parameters: {'min_samples_split': 14, 'max_depth': 230, 'n_estimators': 63}. Best is trial#1 with value: -0.8435614074574971.
[I 2020-07-24 21:17:34,539] Finished trial#4 with value: -0.8385560667365312 with parameters: {'min_samples_split': 9, 'max_depth': 118, 'n_estimators': 103}. Best is 

#### 最適モデルを得る

In [7]:
best_clf = objective.clf(**study.best_params, **objective.fixed_params)

#### スコアで性能評価

In [8]:
best_clf.fit(X_train, y_train)
y_pred_on_test = best_clf.predict(X_test)
print(r2_score(y_test, y_pred_on_test))

0.8658682623468608
